# Tower of Hanoi
Move the tower to the rightmost column. Just keep in mind each column must be sorted by value at all times!

In [ ]:
import asyncio
%pip install ballsort
from ballsort.ballsort_display_utils import open_bs_window
open_bs_window()

In [ ]:
from control_factory import get_ch4_control_sim
bc = get_ch4_control_sim(delay_multiplier=0.1)

In [ ]:
from ch4_scenario import Ch4Scenario
await bc.set_scenario(Ch4Scenario())

In [ ]:
from ball_control import BallControl
from state_update_model import StatePosition

In [ ]:
async def move_ball(bc: BallControl, src: StatePosition, dest: StatePosition):    
    rel_x = src.x - bc.get_position().x
    rel_y = src.y - bc.get_position().y
    await asyncio.gather(
        bc.move_horizontally(rel_x),
        bc.move_vertically(rel_y),
        bc.open_claw())
    await bc.close_claw()
    
    rel_x = dest.x - bc.get_position().x
    rel_y = dest.y - bc.get_position().y
    await asyncio.gather(
        bc.move_horizontally(rel_x),
        bc.move_vertically(rel_y))
    await bc.open_claw()

In [ ]:
async def move_ball_by_column(bc: BallControl, src_x: int, dest_x: int):
    src_column_top_occupied_y = min([ball.pos.y for ball in bc.get_state().balls if ball.pos.x == src_x],default=bc.get_state().max_y)
    dest_column_top_vacant_y = min([ball.pos.y for ball in bc.get_state().balls if ball.pos.x == dest_x],default=bc.get_state().max_y + 1) - 1
    await move_ball(bc=bc, src=StatePosition(x=src_x, y=src_column_top_occupied_y), dest=StatePosition(x=dest_x, y=dest_column_top_vacant_y))

In [ ]:
async def move_tower(height: int, src_x:int, dest_x:int):
    if height == 1:
        await move_ball_by_column(bc=bc, src_x=src_x, dest_x=dest_x)
        return

    intermediate_x = 3 - src_x - dest_x
    await move_tower(height=height - 1, src_x=src_x, dest_x=intermediate_x)
    await move_ball_by_column(bc=bc, src_x=src_x, dest_x=dest_x)
    await move_tower(height=height - 1, src_x=intermediate_x, dest_x=dest_x)

In [ ]:
async def challenge4_solution():    
    await move_tower(height=bc.get_state().max_y + 1, src_x=0, dest_x=2)
    

In [ ]:
await challenge4_solution()